In [ ]:
#79############################################################################
#72#####################################################################
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import plotly.express as px
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import log_loss





pd.set_option('display.max_columns', None)

In [ ]:
filepath = ('C:/Users/andre/OneDrive/Desktop/GitHub/Health-AI/'
            'icr-identify-age-related-conditions/')

In [ ]:
def balanced_log_loss(y_true, y_pred):
    # Someone else in the competition wrote this
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    balanced_log_loss_score = (
        -w0/nc[0]
        * (np.sum(np.where(y_true==0,1,0)
        * np.log(1-y_pred))) - w1/nc[1]
        * (np.sum(np.where(y_true!=0,1,0)
        * np.log(y_pred)))
        ) / (w0+w1)
    return balanced_log_loss_score

In [ ]:
train   = pd.read_csv(filepath+'train.csv')
test    = pd.read_csv(filepath+'test.csv')
greeks  = pd.read_csv(filepath+'greeks.csv')
example = pd.read_csv(filepath+'sample_submission.csv')


def solve_whitespace(df):
    # The files have whitespace issues
    renamer={}
    for col in df.columns:
        renamer[col]=col.strip()
    df.rename(columns=renamer,inplace=True)
    return df
train = solve_whitespace(train)
test  = solve_whitespace(test)

In [ ]:
# Exploring missing values

train_summary = pd.DataFrame(train[1:].dtypes, columns=['data type'])
train_summary['missing'] = train.isnull().sum().values 
desc = pd.DataFrame(train.describe(include='all').transpose())
train_summary[train_summary['missing']!=0]

In [ ]:
def prep_df(df):
    # Move categorical col to beginning of df
    cols = list(df.columns)
    cols.remove('EJ')
    cols.insert(1,'EJ') # Temporarily I will remove this
    df['EJ']=df['EJ'].replace({'A':0,'B':1})
    df=df[cols]
    
    df=df.set_index('Id').copy(deep=True)
    
    """
    7 columns have 1, 2, or 3 null values. If I try to find insights
    from those, I will certainly overfit. I am simply imputing those
    values. Due to my concerns of overfitting due to the tiny dataset,
    I am going to only impute those using the mean/mode. Fancier methods
    risk more overfitting. Columns BQ and EL each have 60 nulls (53 of
    those overlap). For these columns I will create new columns that
    mark them as having had nulls and then impute the missing values.
    """
    df['BQnull'] = 0
    df.loc[df['BQ'].isnull(),'BQnull'] = 1
    df['ELnull'] = 0
    df.loc[df['EL'].isnull(),'ELnull'] = 1
    # Impute values with mean and mode
    for col in df.columns:
        if col in {'Class','Id','BQnull','ELnull'}: continue
        if col=='EJ':
            df[col].fillna(train[col].mode()[0],inplace=True)
        else:
            df[col].fillna(train[col].mean(),inplace=True)
    
    return df
train=prep_df(train)
test=prep_df(test)

In [ ]:
#Scale Columns
num_cols = list(train.columns)
num_cols.remove('Class')
sc = StandardScaler()
train[num_cols] = sc.fit_transform(train[num_cols])
test[num_cols]  = sc.transform(test[num_cols])

In [ ]:
fold_count=5



models=[]
test_preds=[]
oof_preds = np.zeros(len(train))

# Separate into input variables and target variable.
x = train.drop(columns='Class').copy(deep=True)
y = train['Class'].copy(deep=True)
del train

weights = class_weight.compute_sample_weight('balanced', y)

skf = StratifiedKFold(n_splits=fold_count, shuffle=True, random_state=42)
for fold, (train_idx, valid_idx) in enumerate(skf.split(x, y)):
    fold_message = 'Fold '+str(fold+1)+' of '+str(fold_count)+' '
    print(fold_message + '-' * (79-len(fold_message)))

    x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
    x_valid, y_valid = x.iloc[valid_idx], y.iloc[valid_idx]
    
    # Build Model
    model = XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        n_estimators=1000,
        random_state=1,
        verbosity=1,
        n_jobs=-1,
        # learning_rate= 0.005, 
        max_depth=4,
        colsample_bytree= 0.67,
        # subsample= 0.80,
        eta= 0.2,
        early_stopping_rounds=20,
        # gamma= 1.5
    )
    model.fit(
        x_train,
        y_train,
        sample_weight=weights[train_idx],
        eval_set=[(x_valid, y_valid)],
        verbose=1)
    models.append(model)
    
    valid_preds = model.predict_proba(x_valid)[:, 1]
    test_preds.append(model.predict_proba(test)[:, 1])
    
    # Score OOF
    print(
        'Validation OOF balanced_log_loss',
        balanced_log_loss(y_valid, valid_preds))
    print(
        'Validation OOF log_loss with sample_weights',
        log_loss(y_valid, valid_preds, sample_weight=weights[valid_idx]))
    print(
        'Validation OFF log_loss',
          log_loss(y_valid, valid_preds))
    
    oof_preds[valid_idx] = valid_preds
    
test_preds = np.mean(test_preds, axis=0)
x['Pred']=oof_preds
x.sort_index(inplace=True)
y.sort_index(inplace=True)

# Score OOF
print(
    '\n\nCombined OOF balanced_log_loss',
    balanced_log_loss(y, oof_preds))
print(
    'Combined OOF log_loss with sample_weights',
    log_loss(y, oof_preds, sample_weight=weights))
print(
    'Combined OFF log_loss',
      log_loss(y, oof_preds))

In [ ]:
#Confusion Matrix for OOF predictions

cm = confusion_matrix(y, x['Pred'].round())

fig = px.imshow(cm,
                text_auto=True,
                labels=dict(x="Predicted", y="Actual", color="Count"),
                x=['0', '1'],
                y=['0', '1']
               )
fig.update_xaxes(side="top")
fig.update_layout(width=500,height=500)

fig.show()

In [ ]:
# Feature Importance
for model_pos in range(len(models)):
    model_feature_importance = pd.DataFrame(
        {
            'Value':models[model_pos].feature_importances_,
            'Feature':x.drop(columns='Pred').columns})
    model_feature_importance['Model']=model_pos
    if model_pos==0: feature_importance = model_feature_importance
    else: feature_importance = pd.concat(
        [feature_importance,model_feature_importance],axis=0)

# Sort by total importance across all models
total_feature_importance=feature_importance.groupby('Feature')['Value']
total_feature_importance=total_feature_importance.sum().to_frame().reset_index()

total_feature_importance.rename(
    columns={'Value':'Total Value'},
    inplace=True)
feature_importance=feature_importance.merge(
    total_feature_importance,
    how = 'left',
    on = 'Feature')
# I just want to see top 10 and the special features
feature_importance.sort_values(
    by=["Total Value",'Model'],
    inplace=True)
features_to_plot = feature_importance.drop_duplicates('Feature')
features_to_plot = list(features_to_plot.iloc[-10:]['Feature'].values)
for col in [
    'EJ',
    'BQnull',
    'BQ',
    'ELnull',
    'EL'
    ]:
    if col not in features_to_plot:
        features_to_plot.append(col)
feature_importance=feature_importance[
    feature_importance['Feature'].isin(features_to_plot)]
feature_importance.sort_values(by=["Total Value",'Model'],inplace=True)
feature_importance.rename(columns={'Value':'Importance'},inplace=True)


fig = px.bar(feature_importance, x='Importance', y='Feature', facet_col="Model")
fig.update_layout(width=1000,height=500)
fig.show()

In [ ]:
submission = test.copy(deep=True)
submission['class_1'] = test_preds
submission['class_0'] = [1 - x for x in test_preds]
submission=submission[['class_0','class_1']]

# Save the out of fold predictions on the train dataset
x=x['Pred']

In [ ]:
save_output=False

if save_output:
    outputFileName = 'v2'
    outputFileName += (' ' + str(datetime.today())[:22].replace(':','.'))
    x.to_csv(outputFileName + ' ooftrain.csv')
    submission.to_csv(outputFileName + '.csv')

submission.to_csv('submission.csv')
submission